В данном упражнении мы посчитаем рекомендации методом коллаборативной фильтрации Collaborative Filtering.

Мы будем использовать классический датасет MovieLens (1m записей), поэтому необходимо загрузить его с сайта GroupLens

**Повторение теории**

В центре любой рекомендательной системы находится матрица оценок (ratings matrix). По строкам в ней идут пользователи (users), по стоблцам - товары (items), пересечение - оценка товара пользователем (ratings).

<img src=img/user_item.png width=350>

Задача рекомендательной системы:
1. спрогнозировать рейтинг товара для тех пар (user, item), где еще не выставлялись оценки
2. на основании предсказанной матрицы оценок предложить список товаров для рекомендаций каждому пользователю

Суть метода коллаборативной фильтрации: предложить то,что любят клиенты со схожими привычками. Для реокмендации требуется оценить привычки других пользоватлеей системы, отсюда название - "коллаборативная".

Классическая реализация алгоритма использует принцип k-ближайших соседей (kNN). 
- По каждому пользователю ищем топ-k наиболее похожих на него
- Оценки соседей агрегируем (например, путем расчета взвешенного средних оценок)
- Среди тех, которые пользователь еще не оценивал, показываем товары с наилучшим средним ретингом

<img src=img/ub.png width=550>

In [18]:
import pandas
import os
import numpy
data_path = "/Users/Konstantin/HSE/MasterProgram/Практический Семинар/Recommender Systems/RecSys_lesson1/ml-1m/"

Загружаем пользовательские оценки (файл ratings.dat)

In [2]:
ratings = pandas.read_csv(data_path + "ratings.dat", sep="::", names=['user_id','movie_id','rating','ts'])
ratings.head(5)

/Users/Konstantin/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,user_id,movie_id,rating,ts
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


Загружаем справочник фильмов (movies.dat). Он понадобится дальше для вывода названий

In [3]:
movies = pandas.read_csv(data_path + "movies.dat", sep="::", names=['movie_id','title','genre'])

/Users/Konstantin/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


В датасете имеем 1 млн оценок (достаточно неплохо)

In [4]:
ratings.shape

(1000209, 4)

Теперь нужно преобразовать список рейтингов в стандартный матричный формат (user x item => rating). Для этого используем удобный метод pivot().

In [5]:
R = ratings.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)
R.shape

(6040, 3706)

Из справочника фильмов делаем словарь. По идентификаторам будем смортеть названия фильмов.

In [10]:
id2title = dict(zip(movies.index, movies.title))

Нам важно перевести матрицу в сжатое (sparse) представление, иначе при дальнейшем подсчете попарных расстояний рискуем быстро "съесть" всю память! (можете поэкспериментировать)

Для этого подгружаем из библиотеки scipy нужный нам тип csr_matrix

In [11]:
from scipy.sparse import csr_matrix
R = csr_matrix(R.values)
type(R)

scipy.sparse.csr.csr_matrix

Мы можем посчитать матрицу расстояний, например, с помощью метрики cosine_similarity из библиотеки sklearn, а затем вручную отбирать ближайших пользователей...

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(R)

...Но удобнее воспользоваться классом NearestNeighbors из той же библиотеки scikit-learn, который позволяет искать ближашие точки (соседей) по отношению к заданной.

Параметры класса
- algorithm 
  - 'brute' - не преобразуем исходную выборку, а при каждом запросе на поиск полностью сканируем ее (brute-force)
  - 'kd-tree' или 'ball-tree' - строим специальное дерево для ускорения поиска по выборке, дерево хранится в памяти
- metric (какой метрикой мы считаем близость точек)
    - 'cosine' - косинусная близость
    - кроме того можно выбрать из 25 разных метрик
- n_neighbors - количество возвращаемых ближайших соседей

In [13]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10, n_jobs=-1)
model_knn.fit(R)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=-1, n_neighbors=10, p=2, radius=1.0)

Обратим внимание, что посколько мы выбрали алгоритм 'brute', при вызове метода fit() никакие расстояния не рассчитываются, а просто подгружается матрица рейтингов R. Рассчитываться они будут по необходимости.

Предположим, мы генерируем рекомендации для пользователя user_id = 0 и на его домашней странице хотим показать ему n_neighbors = 10 рекомендаций

In [14]:
current_user_id=0
n_neighbors = 10

Найти ближайших соседей можно методом kneighbors()

In [15]:
distances, neighbors = model_knn.kneighbors(R[current_user_id], n_neighbors=n_neighbors+1)
neighbors[0]

array([   0, 5342, 5189, 1480, 1282, 5704, 5761, 1358, 1475,  540, 1857])

Обратим внимание, что при использовании метода kneighbors() в списке соседей возвращается и сам пользователь. Расстояние до себя всегда равно 0, поэтому любой пользователь всегда вяляется ближайшим соседом к себе.

Если бы мы искали соседей ближайших к произвольно выбранной точке, нам бы такая логика подошла, но поскольку мы ищем соседей конкретного пользователя, давайте список соседей отфильтруем:

In [16]:
neighbors = neighbors[0][1:]
distances = distances[0][1:]

Посмотрим, сколько рейтингов проставили "соседи"

In [19]:
numpy.sum(R[neighbors], axis=1)

matrix([[ 167.],
        [ 169.],
        [ 438.],
        [ 154.],
        [ 640.],
        [1132.],
        [ 350.],
        [ 263.],
        [ 606.],
        [ 388.]])

Самый простой способ агрегировать рейтинги от нескольких соседей - усреднить их. 

Ставим правильный axis, ведь нам нужно усреднять по стоблцам (по стоблцам у нас отложены фильмы).

In [20]:
import numpy
predicted_ratings = numpy.mean(R[neighbors].todense(), axis=0)

А ещё лучше не усреднить, а взвесить по расстояниям (будем давать больший вес тем пользователям, кто ближе). В numpy для этого есть удобная функция average().

In [ ]:
# predicted_ratings = numpy.average(R[neighbors].todense(), weights=distances, axis=0)

Теперь отсортируем фильмы в порядке убывания среднего рейтинга

In [278]:
recommended_films_ids = sorted(zip(predicted_ratings, range(len(predicted_ratings))), key = lambda x: -x[0])

['Toy Story (1995)',
 'Brady Bunch Movie, The (1995)',
 'Junior (1994)',
 'Hour of the Pig, The (1993)',
 "I Don't Want to Talk About It (De eso no se habla) (1993)",
 'African Queen, The (1951)',
 'Farewell to Arms, A (1932)',
 'Unhook the Stars (1996)',
 'King of New York (1990)',
 'Metropolitan (1990)']

Выберем топ-10  рекомендаций и выпишем названия рекомендуемых фильмов

In [ ]:
[id2title[x[1]] for x in recommended_films_ids if x[0] > 0.0][0:10]

**Минусы:**

При таком подходе нам нужно считать матрицу расстояний! Попробуйте проделать данное упражнение не с 6000 пользователей, с 1 млн. В итоге получится матрица расстояний с  10^12 элементов, а это примерно 8 терабайт данных. Есть способы как можно частично ускорить, но все они требуют затрат. 

Если есть требование считать быстро, рекомендуется сделать выбор в пользу матричных разложений. Они используют те же данные (матрицу оценок), но считаются в разы эфективнее.

Полный текст кода

In [ ]:
import pandas
import os
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

# Load Data
data_path = "/Users/Konstantin/HSE/MasterProgram/Практический Семинар/Recommender Systems/RecSys_lesson1/ml-1m/"
ratings = pandas.read_csv(data_path + "ratings.dat", sep="::", names=['user_id','movie_id','rating','ts'])
movies = pandas.read_csv(data_path + "movies.dat", sep="::", names=['movie_id','title','genre'])

# Format Data
R = ratings.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)
id2title = dict(zip(movies.index, movies.title))
R = csr_matrix(R.values)

# Create Nearest Neghbor Model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10, n_jobs=-1)
model_knn.fit(R)

# Get Neighbors 
current_user_id=0
n_neighbors = 10
distances, neighbors = model_knn.kneighbors(R[current_user_id], n_neighbors=n_neighbors+1)

neighbors = neighbors[0][1:]
distances = distances[0][1:]

import numpy
predicted_ratings = numpy.mean(R[neighbors].todense(), axis=0)

recommended_films_ids = sorted(zip(predicted_ratings, range(len(predicted_ratings))), key = lambda x: -x[0])

[id2title[x[1]] for x in recommended_films_ids if x[0] > 0.0][0:10]